In [ ]:
import numpy as np
   
    
    
def matvec(A,B,C,x):
    res = x.copy()
    for i in range(1,len(A)-1):
        res[i] = A[i].dot(x[i])+B[i-1].dot(x[i-1])+C[i].dot(x[i+1])
    res[0] = A[0].dot(x[0])+C[0].dot(x[1])
    res[-1] = B[-1].dot(x[-2])+A[-1].dot(x[-1])
    return res
    

def solve_linear(A,B,C,d):
    x = np.empty(d.shape,dtype= np.float64)
    Y = np.empty(d.shape,dtype= np.float64)
    gamma = np.empty(C.shape,dtype= np.float64)
    ialpha= inv(A[0])
    gamma[0] = ialpha.dot(C[0])
    Y[0] = ialpha.dot(d[0])
    for i in range(1,len(Y)-1):
        ialpha = inv(A[i]-B[i-1].dot(gamma[i-1]))
        gamma[i] = ialpha.dot(C[i])
        Y[i] = ialpha.dot(d[i]-B[i-1].dot(Y[i-1]))

    ialpha = inv(A[len(Y)-1]-B[len(Y)-2].dot(gamma[len(Y)-2]))
    Y[len(Y)-1] = ialpha.dot(d[len(Y)-1]-B[len(Y)-2].dot(Y[len(Y)-2]))
    x[len(Y)-1] = Y[len(Y)-1]
    for i in reversed(range(len(Y)-1)):
        x[i] = Y[i]-gamma[i].dot(x[i+1])
    return x

    
def newton_solver(grid,x0,r):
    x = x0.copy()
    A,B,C,d = generate_problem(grid,x0)
    d += r
    for i in range(max_iter):
        x += solve_linear(A,B,C,d)
        A,B,C,d = generate_problem(grid,x)
        d += r
        if(np.abs(matvec(A,B,C,x)-b).mean() < eps):
            break        
    return x    


def solver(grid,x0,r):
    max_iter=1000
    if(h >= max_h):
        return newton_solver(grid,x0,r)
    else:
        x = x0.copy()
        for i in range(max_iter):
            x = multigrid(grid,x,r)
            if(np.abs(p.calc(x)-b).mean() < eps):
                return x
    
    
def interpolate(in_grid,x,out_grid):
    
    

def generate_problem(grid,x):
    
    
    
def generate_grid(h):


    
def multigrid(grid,v,r):
    grid2 = generate_grid(2*h)
    v2 = interpolate(grid,v,grid2)
    A,B,C,d = generate_problem(grid2,v2)
    r2 = interpolate(grid,r,grid2)
    u2 = solve(A2,v2,grid2,matvec(A,B,C,v2)-d+r2)
    u = v+interpolate(grid,u2-v2,grid2)
    return u
    
    